In [1]:
import os 
from typing import Annotated
from openai import AsyncOpenAI
import pandas as pd
from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

In [2]:
# Cargar el archivo CSV
df = pd.read_csv('Database/BloodPressuredataset.csv')

# Mostrar las primeras 5 filas
print("Primeras filas del archivo:")
print(df.head())

# Mostrar información general de las columnas
print("\nInformación de las columnas:")
print(df.info())

# Estadísticas básicas de las columnas numéricas
print("\nEstadísticas descriptivas:")
print(df.describe())

FileNotFoundError: [Errno 2] No such file or directory: 'Database/BloodPressuredataset.csv'

In [ ]:
import nest_asyncio
import pandas as pd
import numpy as np
from typing import Annotated
from semantic_kernel.functions import kernel_function
from semantic_kernel.skill_definition import sk_function, SKFunctionContext



class HealthDataPlugin:
    """Proporciona funciones basadas en datos médicos de pacientes."""

    def __init__(self, csv_path: str):
        # Cargar el dataset de pacientes desde un archivo CSV
        self.df = pd.read_csv(csv_path)

    @kernel_function(description="Devuelve el promedio de presión sistólica de todos los pacientes.")
    def get_avg_systolic_pressure(self) -> Annotated[float, "Promedio de presión sistólica"]:
        return float(self.df["Presion_Sistolica"].mean())

    @kernel_function(description="Devuelve la cantidad de pacientes con IMC mayor a 30 (obesidad).")
    def get_obese_patient_count(self) -> Annotated[int, "Cantidad de pacientes con IMC alto"]:
        return int((self.df["IMC"] > 30).sum())
    
    @kernel_function(description="Calcula presión sistólica y diastólica basada en edad, IMC y glucosa.")
    def calculate_pressure(self, age: int, bmi: float, glucose: float) -> Annotated[str, "Presión sistólica y diastólica estimada"]:
        base_sys, base_dia = 120, 80
        systolic_bp = base_sys + 0.4*(age-30) + 1.5*(bmi-22.5) + 0.03*(glucose-90)
        diastolic_bp = base_dia + 0.25*(age-30) + 1.0*(bmi-22.5) + 0.02*(glucose-90)
        return f"Sistólica: {int(systolic_bp)} mmHg, Diastólica: {int(diastolic_bp)} mmHg"

    @kernel_function(description="Devuelve la edad promedio de los pacientes.")
    def get_avg_age(self) -> Annotated[float, "Edad promedio de los pacientes"]:
        return float(self.df["Edad"].mean())
    
    @kernel_function(description="Devuelve la edad promedio de los pacientes.")
    def get_avg_age(self) -> Annotated[float, "Edad promedio de los pacientes"]:
        return float(self.df["Edad"].mean())
    
    @sk_function(description="Genera datos sintéticos de presión arterial.")
    def generate_data(self, context: SKFunctionContext) -> str:
        data = {
            'Edad': np.random.randint(18, 80, 100).tolist(),
            'Presión Sistólica': np.random.randint(110, 160, 100).tolist(),
            'Presión Diastólica': np.random.randint(70, 100, 100).tolist()
        }
        df = pd.DataFrame(data)
        path = "blood_pressure_data.csv"
        df.to_csv(path, index=False)
        return f"Datos guardados en {path}"


df_demo = pd.DataFrame({
    "Edad": [25, 45, 65],
    "Presion_Sistolica": [120, 135, 145],
    "IMC": [22, 31, 29],
    "Glucosa": [90, 110, 85]
})
os.makedirs("data", exist_ok=True)
csv_path = "data/demo_pacientes.csv"
df_demo.to_csv(csv_path, index=False)

# 📦 Instancia y ejecución
plugin = HealthDataPlugin(csv_path)
print("Promedio presión sistólica:", plugin.get_avg_systolic_pressure())
print("Pacientes con IMC > 30:", plugin.get_obese_patient_count())
print("Presión estimada para paciente de 50 años, IMC 28, glucosa 100:", plugin.calculate_pressure(50, 28, 100))
print("Edad promedio:", plugin.get_avg_age())



In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [ ]:
health_plugin = HealthDataPlugin("Database/BloodPressuredataset.csv")

agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[health_plugin],
    name="VitalMind",
    instructions=(
       "Eres VitalMind, un agente experto en salud. "
        "Puedes acceder a información médica real de pacientes desde un archivo CSV. "
        "Responde de manera clara, empática y útil a las preguntas del usuario. "
        "Además, ofrece explicaciones detalladas y recomendaciones personalizadas cuando sea posible, "
        "utilizando datos clínicos disponibles como presión arterial, glucosa, IMC y ritmo cardíaco. "
        "Si se requiere, puedes invocar plugins para análisis, predicciones o clustering de pacientes. "
        "Recuerda priorizar el bienestar emocional del usuario en cada respuesta."
    )
)

In [ ]:
async def main(agent):
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Tengo 75 años y me han subido los niveles de glucosa, ¿qué puedo hacer?",
        "Mi padre tiene 82 años y a veces se le olvida tomar sus medicamentos, ¿cómo puedo ayudarlo?",
        "¿Qué debo hacer si tengo presión arterial alta y me siento mareado?",
    ]

    for user_input in user_inputs:
        print(f"\n🧑‍⚕️ User: {user_input}\n")
        first_chunk = True
        try:
            async for response in agent.invoke_stream(messages=user_input, thread=thread):
                if first_chunk:
                    agent_name = getattr(response, "name", "Agente")
                    print(f"🤖 {agent_name}: ", end="", flush=True)
                    first_chunk = False
                print(getattr(response, "content", str(response)), end="", flush=True)
                thread = getattr(response, "thread", thread)
            print()
        except Exception as e:
            print(f"⚠️ Error al procesar la respuesta: {e}")

    if thread and hasattr(thread, "delete"):
        await thread.delete()

# Si tenés tu agente ya creado:
# await main(agent)


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

ai_agent_settings = AzureAIAgentSettings.create()


# Simulación de agente conversacional
async def main():
    user_inputs = [
        "¿Cuál es la presión promedio?",
        "¿Cuál es la frecuencia cardíaca promedio?",
    ]

    outputs = []
    
    for user_input in user_inputs:
        outputs.append(f"# User: '{user_input}'")
        if "presión" in user_input:
            outputs.append("# Agent: La presión sistólica promedio es 123.4 mmHg.")
        else:
            outputs.append("# Agent: Lo siento, pero no tengo la información para responder esa pregunta.")
    
    outputs.append("\nCleaned up agent, thread, file, and vector store.")
    
    for line in outputs:
        print(line)

await main()
